In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("../Ex_Files_NLP_Python_ML_EssT/Ex_Files_NLP_Python_ML_EssT/Exercise Files/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

body_len  punct%  0  1  2  3  4  5  6  7  ...  8094  8095  8096  8097  \
0       128     4.7  0  0  0  0  0  0  0  0  ...     0     0     0     0   
1        49     4.1  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2        62     3.2  0  0  0  0  0  0  0  0  ...     0     0     0     0   
3        28     7.1  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4       135     4.4  0  0  0  0  0  0  0  0  ...     0     0     0     0   

   8098  8099  8100  8101  8102  8103  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 8106 columns]

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
rf = RandomForestClassifier()
param = {'n_estimators': [100,200,400], 'max_depth':[30,60,90,None]}

gs = GridSearchCV(rf,param,cv=5,n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
7       38.442592      0.752321         0.465355        0.088271   
11      67.086046      4.587633         0.405514        0.091275   
8       75.103160      1.144627         0.635696        0.061575   
10      41.769276      1.178104         0.436230        0.046432   
9       21.793802      0.657172         0.345074        0.063784   

   param_max_depth param_n_estimators  \
7               90                200   
11            None                400   
8               90                400   
10            None                200   
9             None                100   

                                      params  split0_test_score  \
7     {'max_depth': 90, 'n_estimators': 200}           0.977558   
11  {'max_depth': None, 'n_estimators': 400}           0.976661   
8     {'max_depth': 90, 'n_estimators': 400}           0.977558   
10  {'max_depth': None, 'n_estimators': 200}           0.976661   
9   {'max_depth': None, 'n_estimators': 100}           0.979354   

    split1_test_score  split2_test_score  split3_test_score  \
7            0.976661           0.974843           0.966757   
11           0.978456           0.974843           0.968553   
8            0.975763           0.975741           0.966757   
10           0.976661           0.975741           0.965858   
9            0.973968           0.975741           0.966757   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
7            0.974843         0.974132        0.003835                1  
11           0.972147         0.974132        0.003483                2  
8            0.972147         0.973593        0.003844                3  
10           0.973046         0.973593        0.004088                3  
9            0.971249         0.973414        0.004243                5

In [5]:
rf = RandomForestClassifier()
param = {'n_estimators': [100,200,400], 'max_depth':[30,60,90,None]}

gs = GridSearchCV(rf,param,cv=5,n_jobs=-1)
gs_fit = gs.fit(X_count_feat,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
9       22.206097      0.908318         0.337296        0.025772   
8       74.369318      0.660105         0.601388        0.047694   
7       38.913927      1.332640         0.477321        0.050278   
11      66.133002      4.561886         0.424053        0.059263   
6       19.976070      0.620759         0.320940        0.025649   

   param_max_depth param_n_estimators  \
9             None                100   
8               90                400   
7               90                200   
11            None                400   
6               90                100   

                                      params  split0_test_score  \
9   {'max_depth': None, 'n_estimators': 100}           0.976661   
8     {'max_depth': 90, 'n_estimators': 400}           0.977558   
7     {'max_depth': 90, 'n_estimators': 200}           0.979354   
11  {'max_depth': None, 'n_estimators': 400}           0.978456   
6     {'max_depth': 90, 'n_estimators': 100}           0.976661   

    split1_test_score  split2_test_score  split3_test_score  \
9            0.975763           0.973944           0.967655   
8            0.975763           0.973944           0.968553   
7            0.973968           0.973944           0.968553   
11           0.971275           0.973944           0.969452   
6            0.973070           0.975741           0.965858   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
9            0.972147         0.973234        0.003192                1  
8            0.969452         0.973054        0.003511                2  
7            0.968553         0.972875        0.004041                3  
11           0.969452         0.972516        0.003395                4  
6            0.969452         0.972156        0.004024                5